In [ ]:
# Import Libraries
import pandas as pd
import os

from snowflake.snowpark import Session

In [ ]:
# Connection Parameters to establish Snowflake connection
connection_parameters = {
    "user":"KRISHNAOV",
    "password":"ovAIIntern@09/05",
    "account":"kyb42073.us-east-1",
    "warehouse":"OPENVIEW",
    "database":"COMPANYRECSDB_TEST",
    "schema":"PUBLIC",
    "role":"ACCOUNTADMIN"
}

# Establish a session with Snowflake
session = Session.builder.configs(connection_parameters).create()

In [ ]:
# Path to the large CSV file and the local directory to store chunks
file_path = '<path_to_csv_file>'
local_dir = '<directory_to_store_chunks>'

# Define the number of rows to use for the schema sample and the chunk size for reading the CSV
sample_size = 1000
chunk_size = 100000  # Size of each chunk to read i.e number of rows

stage_name = '<stage_name'
table_name = 'table_name'

In [ ]:
# Read a sample of the CSV file to infer schema
sample_df = pd.read_csv(file_path, nrows=sample_size)
sf_sample_df = session.create_dataframe(sample_df)

# Create a table with the inferred schema if it doesn't exist
table_exists_query = f"SELECT COUNT(*) AS CNT FROM information_schema.tables WHERE table_schema = '{connection_parameters['schema']}' AND table_name = '{table_name}'"
table_exists_result = session.sql(table_exists_query).collect()

# Create a table with the inferred schema if it doesn't exist
if table_exists_result[0]['CNT'] == 0:
    sf_sample_df.write.mode('overwrite').save_as_table(table_name)

In [ ]:
# The number of rows to skip is set to the sample size for the first chunk
skiprows = sample_size

# Check if the stage exists and create it if not
create_stage_query = f"CREATE STAGE IF NOT EXISTS {stage_name} FILE_FORMAT = (TYPE = 'CSV')"
session.sql(create_stage_query).collect()

In [ ]:
# Iterate over the CSV file in chunks, skipping the sample rows in the first chunk
for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size, skiprows=range(1, skiprows))):
    # Update skiprows to None for subsequent chunks
    skiprows = None
    
    # Define a unique file name for each chunk
    file_name = f"chunk_{i}.csv"
    local_file_path = os.path.join(local_dir, file_name)
    
    # Save the chunk locally
    chunk.to_csv(local_file_path, index=False)
    
    # Upload the file to the stage with auto compression
    put_command = f"""
    PUT file://{local_file_path} @{stage_name} 
    AUTO_COMPRESS=TRUE
    """
    session.sql(put_command).collect()
    
    # Copy the data into the Snowflake table
    copy_command = f"""
        COPY INTO {table_name}
        FROM @{stage_name}
        FILE_FORMAT = (TYPE = 'CSV' COMPRESSION = 'GZIP' FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER = 1)
        ON_ERROR = 'CONTINUE'
    """
    session.sql(copy_command).collect()
    
    # Optionally, remove the local file after it's uploaded
    os.remove(local_file_path)

In [ ]:
# Close the session
session.close()